# Libary

In [ ]:
!pip install bitstring

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import Counter
from math import log, ceil
from bitstring import BitArray, Bits
import os
import heapq
import time

#HUFFMAN

In [ ]:
import os
import heapq
import time
from PIL import Image
import re
import sys
import re
import numpy as np

def read_image_to_string(file_path):
    my_string = np.asarray(Image.open(file_path),np.uint8)
    shape = my_string.shape
    print ("Enetered string is:",my_string)
    stringToEncode = str(my_string.tolist())
    s1 = stringToEncode
    return s1,shape

class BinaryTree:
    def __init__(self, value, frequ):
        self.value = value
        self.frequ = frequ
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequ < other.frequ

    def __eq__(self, other):
        return self.frequ == other.frequ


class Huffmancode_img:
    def __init__(self, path):
        self.path = path
        self.__heap = []
        self.__code = {}
        self.__reversecode = {}

    def __frequency_from_text(self, text):
        frequ_dict = {}
        for char in text:
            if char not in frequ_dict:
                frequ_dict[char] = 0
            frequ_dict[char] += 1
        return frequ_dict

    def __Build_heap(self, frequency_dict):
        for key in frequency_dict:
            frequency = frequency_dict[key]
            binary_tree_node = BinaryTree(key, frequency)
            heapq.heappush(self.__heap, binary_tree_node)

    def __Build_Binary_Tree(self):
        while len(self.__heap) > 1:
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            sum_of_freq = binary_tree_node_1.frequ + binary_tree_node_2.frequ
            newnode = BinaryTree(None, sum_of_freq)
            newnode.left = binary_tree_node_1
            newnode.right = binary_tree_node_2
            heapq.heappush(self.__heap, newnode)

    def __Build_Tree_Code_Helper(self, root, curr_bits):
        if root is None:
            return
        if root.value is not None:
            self.__code[root.value] = curr_bits
            self.__reversecode[curr_bits] = root.value
            return
        self.__Build_Tree_Code_Helper(root.left, curr_bits + '0')
        self.__Build_Tree_Code_Helper(root.right, curr_bits + '1')

    def __Build_Tree_Code(self):
        root = heapq.heappop(self.__heap)
        self.__Build_Tree_Code_Helper(root, '')

    def __Build_Encoded_Text(self, text):
        encoded_text = ''
        for char in text:
            encoded_text += self.__code[char]
        return encoded_text

    def __Build_Padded_Text(self, encoded_text):
        padding_value = 8 - (len(encoded_text) % 8)
        for i in range(padding_value):
            encoded_text += '0'
        padded_info = "{0:08b}".format(padding_value)
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text

    def __Build_Byte_Array(self, padded_text):
        array = []
        for i in range(0, len(padded_text), 8):
            byte = padded_text[i:i + 8]
            array.append(int(byte, 2))
        return array

    def compression_img(self):
        filename, file_extension = os.path.splitext(self.path)
        output_path = 'encoded_huffman1.txt'
        # with open(self.path, 'r') as file, open(output_path, 'wb') as output:
        with open(output_path, 'wb') as output:
            # text = file.read()
            # text = text.rstrip()
            text, shape = read_image_to_string(self.path)
            frequency_dict = self.__frequency_from_text(text)
            self.__Build_heap(frequency_dict)
            self.__Build_Binary_Tree()
            self.__Build_Tree_Code()
            encoded_text = self.__Build_Encoded_Text(text)
            print('encoded_text ', encoded_text[0:50])
            padded_text = self.__Build_Padded_Text(encoded_text)
            bytes_array = self.__Build_Byte_Array(padded_text)
            print('bytes_array ', bytes_array[0:50])
            final_bytes = bytes(bytes_array)
            output.write(final_bytes)
        print("Compressed")
        return output_path, shape, text, encoded_text

    def __Remove_Padding(self, text):
        padded_info = text[:8]
        extra_padding = int(padded_info, 2)
        text = text[8:]
        padding_removed_text = text[:-1 * extra_padding]
        return padding_removed_text

    def __Decompress_Text(self, text):
        decoded_text = ''
        current_bits = ''
        for bit in text:
            current_bits += bit
            if current_bits in self.__reversecode:
                character = self.__reversecode[current_bits]
                decoded_text += character
                current_bits = ""
        return decoded_text

    def decompress_img(self, input_path, shape):
        filename, file_extension = os.path.splitext(input_path)
        output_path = 'decoded_huffman.txt'
        with open(input_path, 'rb') as file:
            bit_string = ''
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = file.read(1)
            actual_text = self.__Remove_Padding(bit_string)
            decompressed_text = self.__Decompress_Text(actual_text)
            # output.write(decompressed_text)
        print("Decompressed")
        temp = re.findall(r'\d+', decompressed_text)
        res = list(map(int, temp))
        res = np.array(res)
        res = res.astype(np.uint8)
        res = np.reshape(res, shape)
        #print(res)
        print("Observe the shapes and input and output arrays are matching or not")
        print("Input image dimensions:",shape)
        print("Output image dimensions:",res.shape)
        data = Image.fromarray(res)
        data.save('uncompressed_huffman.jpg')
        # return decompressed_text



In [ ]:
path = "/content/image (1).jpg"
h = Huffmancode_img(path)

t0 = time.time()
output_path, shape, text, encoded_text= h.compression_img()
t1 = time.time()
total_1 = t1 - t0

t2 = time.time()
h.decompress_img(output_path, shape)
t3 = time.time()
total_2 = t3 - t2

Enetered string is: [[[220 219 217]
  [220 219 217]
  [219 218 216]
  ...
  [216 211 208]
  [215 210 207]
  [215 210 207]]

 [[193 192 190]
  [193 192 190]
  [192 191 189]
  ...
  [179 174 171]
  [178 173 170]
  [178 173 170]]

 [[200 199 197]
  [200 199 197]
  [200 199 197]
  ...
  [182 177 174]
  [181 176 173]
  [181 176 173]]

 ...

 [[ 47  29   5]
  [ 55  35  11]
  [ 65  43  19]
  ...
  [196 196 196]
  [194 194 194]
  [188 188 188]]

 [[ 43  25   3]
  [ 49  31   7]
  [ 60  38  15]
  ...
  [197 197 197]
  [196 196 196]
  [189 189 189]]

 [[ 39  23   0]
  [ 46  28   6]
  [ 56  36  12]
  ...
  [197 197 197]
  [196 196 196]
  [190 190 190]]]
encoded_text  10011001100111110111101101101001111010111101010011
bytes_array  [8, 153, 159, 123, 105, 235, 212, 245, 214, 36, 251, 219, 79, 94, 167, 174, 177, 39, 215, 169, 235, 196, 245, 134, 36, 250, 245, 61, 120, 158, 176, 196, 159, 94, 167, 175, 19, 214, 24, 147, 235, 196, 245, 210, 122, 253, 137, 62, 188, 79]
Compressed
Decompressed
Observe th

In [ ]:
uncompressed_size = len(text) * 7
compressed_size = len(encoded_text)
r = uncompressed_size/compressed_size
print("Your original file size was", uncompressed_size,"(bits). \nThe compressed size is:",compressed_size,"(bits).")
print("This is a saving of ",uncompressed_size - compressed_size,"(bits).")
r = uncompressed_size/compressed_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Compressed file generated as compressed.txt")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Decoding.......")
print("Thời gian giải nén:", round(total_2,5), "(giây).")


Your original file size was 43892792 (bits). 
The compressed size is: 19024616 (bits).
This is a saving of  24868176 (bits).
Tỷ số nén: 2.30716
Tỷ lệ nén: 43.34 %
Hiệu suất nén: 56.66 %
Compressed file generated as compressed.txt
Thời gian nén: 4.3036 (giây).
Decoding.......
Thời gian giải nén: 5.44692 (giây).


# **LZW**

In [ ]:
import struct
def save_binary_file(data, file_path):
    with open(file_path, 'wb') as f:
        f.write(struct.pack(f'{len(data)}I', *data))


def read_binary_file(file_path):
    with open(file_path, 'rb') as f:
        # Read the binary data from the file
        binary_data = f.read()

        # Unpack the binary data into a list
        data_length = len(binary_data) // struct.calcsize('I')
        data = struct.unpack(f'{data_length}I', binary_data)

        return list(data)

In [ ]:
from collections import defaultdict
from PIL import Image
import numpy as np
import re
import sys

def read_image_to_string(file_path):
    my_string = np.asarray(Image.open(file_path),np.uint8)
    shape = my_string.shape
    print ("Enetered string is:",my_string)
    stringToEncode = str(my_string.tolist())
    s1 = stringToEncode
    return s1,shape
def encoding_img(s1):
    # with open(file_path, 'r') as f:
    #     s1 = f.read()

    # size_in_bytes = sys.getsizeof(my_string)
    table = defaultdict(int)
    for i in range(256):
        ch = chr(i)
        table[ch] = i
    p = s1[0]
    code = 256
    output_code = []
    for i in range(len(s1)):
        if i != len(s1) - 1:
            c = s1[i + 1]
        if p + c in table:
            p = p + c
        else:
            output_code.append(table[p])
            #print(p, table[p])
            table[p + c] = code
            code += 1
            p = c
        c = ""
    output_code.append(table[p])
    #print(p, table[p])
    # print(output_code)
    #print(output_string)
    #print('encode: ',(len(s1) * 8 - len(output_string)*8)/(len(s1) * 8))
    input_size = len(s1)
    return output_code

def decoding_img(op,shape, file_path):
    table = defaultdict(str)
    for i in range(256):
        ch = chr(i)
        table[i] = ch
    old = op[0]
    s = table[old]
    c = s[0]
    count = 256
    # with open(file_path, 'w') as f:
    #     f.write(s)
    output = s
    for i in range(len(op) - 1):
        n = op[i + 1]
        if n not in table:
            s = table[old]
            s += c
        else:
            s = table[n]
        output += s
        c = s[0]
        table[count] = table[old] + c
        count += 1
        old = n
    # with open(file_path, 'r') as f:
    temp = re.findall(r'\d+', output)
    res = list(map(int, temp))
    res = np.array(res)
    res = res.astype(np.uint8)
    res = np.reshape(res, shape)
    #print(res)
    print("Observe the shapes and input and output arrays are matching or not")
    print("Input image dimensions:",shape)
    print("Output image dimensions:",res.shape)
    data = Image.fromarray(res)
    data.save('uncompressed.bmp')

In [ ]:
#  ảnh
inputfile_path="/content/image (1).jpg"
s1,shape = read_image_to_string(inputfile_path)
t0 = time.time()
output_code =encoding_img(s1)
# tinh size
arr_output_code = np.asarray(output_code,np.uint8)
output_string = " ".join(map(str, arr_output_code.tolist()))
uncompressed_size = len(s1) * 7
compressed_size = len(output_string) * 7
t1 = time.time()
total_1 = t1-t0
save_binary_file(output_code, 'output.txt')
t2 = time.time()
Decode_code = read_binary_file('output.txt')
decoding_img(Decode_code,shape,'decode_lzw.txt')
t3 = time.time()
total_2 = t3-t2

Enetered string is: [[[220 219 217]
  [220 219 217]
  [219 218 216]
  ...
  [216 211 208]
  [215 210 207]
  [215 210 207]]

 [[193 192 190]
  [193 192 190]
  [192 191 189]
  ...
  [179 174 171]
  [178 173 170]
  [178 173 170]]

 [[200 199 197]
  [200 199 197]
  [200 199 197]
  ...
  [182 177 174]
  [181 176 173]
  [181 176 173]]

 ...

 [[ 47  29   5]
  [ 55  35  11]
  [ 65  43  19]
  ...
  [196 196 196]
  [194 194 194]
  [188 188 188]]

 [[ 43  25   3]
  [ 49  31   7]
  [ 60  38  15]
  ...
  [197 197 197]
  [196 196 196]
  [189 189 189]]

 [[ 39  23   0]
  [ 46  28   6]
  [ 56  36  12]
  ...
  [197 197 197]
  [196 196 196]
  [190 190 190]]]
Observe the shapes and input and output arrays are matching or not
Input image dimensions: (600, 600, 3)
Output image dimensions: (600, 600, 3)


In [ ]:
r = uncompressed_size/compressed_size
print("Your original file size was", uncompressed_size,"(bits). \nThe compressed size is:",compressed_size,"(bits).")
print("This is a saving of ",uncompressed_size - compressed_size,"(bits).")
r = uncompressed_size/compressed_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Compressed file generated as compressed.txt")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Decoding.......")

Your original file size was 43892792 (bits). 
The compressed size is: 13364360 (bits).
This is a saving of  30528432 (bits).
Tỷ số nén: 3.28432
Tỷ lệ nén: 30.45 %
Hiệu suất nén: 69.55 %
Compressed file generated as compressed.txt
Thời gian nén: 3.61386 (giây).
Decoding.......


#SHANNON

In [ ]:
import os
import heapq
import time
from PIL import Image
import re
import sys
import re
import numpy as np

def read_image_to_string(file_path):
    my_string = np.asarray(Image.open(file_path),np.uint8)
    sudhi = my_string
    shape = my_string.shape
    print ("Enetered string is:",my_string)
    stringToEncode = str(my_string.tolist())
    s1 = stringToEncode
    return s1,shape

In [ ]:
hex2bin = dict('{:x} {:04b}'.format(x,x).split() for x in range(16))
def get_bincode(number, places=""):
    if number == 0.0:
      return "0"*places
    hx = float(number).hex()
    p = hx.index('p')
    bn = ''.join(hex2bin.get(char, char) for char in hx[2:p])
    bn = (bn.strip('0'))
    whole, flt = bn.split(".")
    if places == "":
      places = len(flt)+1
    num = int(hx[p+2:])
    whole = "0" * (num - 1) + whole
    output = whole + flt
    if len(output) < places:
      output += "0" * (places - len(output))
    return output[:places]

def generate(char, total, _dict):
  # calculate probability
  probability = [(x[0]/total) for x in char]

  # calculate codework length
  code_length = [(ceil(-log(x,2))) for x in probability]

  # cumulative probability
  cumulative_probability = 0.0
  for i in range(len(code_length)):
    bin_code = get_bincode(cumulative_probability, code_length[i])
    _dict[char[i][1]] = bin_code
    cumulative_probability += probability[i]
  return _dict

In [ ]:
#  Encode
input_path = "/content/sample_640×426.bmp"
output = "encoded_shannon.txt"
dict_name = "_dict_shannon.csv"

t0 = time.time()
# Read file
content, shape = read_image_to_string(input_path)

# Count character
count = Counter(content)
total = sum(count.values())
char = [x[::-1] for x in list(count.items())]
char.sort(reverse=True)

# Create dictionary
_dict = dict()
generate(char,total,_dict)

# Encode
code = ''
for c in content:
  code += _dict[c]
code_bin = BitArray(bin = code)
max_bin = len(code)

# Save file
# output
with open(output, "wb") as f:
    code_bin.tofile(f)
# dict
keys = []
values = []
for key in _dict:
  keys.append(_dict[key])
  values.append(key)
encoded = {'keys': list(keys), 'values': list(values)}
df = pd.DataFrame(encoded)
df.to_csv(dict_name, index=False)
t1 = time.time()
total_1 = t1-t0

Enetered string is: [[[178 130 144]
  [173 125 139]
  [170 122 138]
  ...
  [125  93 117]
  [124  88 114]
  [122  86 112]]

 [[178 130 144]
  [175 127 141]
  [173 125 141]
  ...
  [127  93 118]
  [125  89 115]
  [123  87 113]]

 [[178 130 144]
  [177 129 143]
  [176 128 144]
  ...
  [126  92 117]
  [125  89 115]
  [127  88 115]]

 ...

 [[  0   1   0]
  [ 17  19  16]
  [  3   2   0]
  ...
  [ 14  24   0]
  [ 67  78  10]
  [ 93 104  36]]

 [[  4   6   1]
  [  7   9   4]
  [  9   8   6]
  ...
  [ 55  64   0]
  [ 26  37   0]
  [ 92 103  35]]

 [[  4   6   1]
  [  7   9   4]
  [  9   8   6]
  ...
  [ 91 100  33]
  [ 44  53   0]
  [115 126  58]]]


In [ ]:
#  Decode
decode_file = "decoded_shannon.txt"

t2 = time.time()
# Read dictionary
df = pd.read_csv(dict_name,dtype=str)
keys = list(df['keys'])
values = list(df['values'])
_dict = dict(zip(keys,values))
with open(output,'rb') as f:
  data = Bits(f)


# Decode
string = ''
s = ''
for c in data.bin[:max_bin]:
  if s+c not in _dict:
    s+=c
  else:
    string+=_dict[s+c]
    s = ''

# Save file
temp = re.findall(r'\d+', string)
res = list(map(int, temp))
res = np.array(res)
res = res.astype(np.uint8)
res = np.reshape(res, shape)
#print(res)
print("Observe the shapes and input and output arrays are matching or not")
print("Input image dimensions:",shape)
print("Output image dimensions:",res.shape)
data = Image.fromarray(res)
data.save('uncompressed_shannon.jpg')
t3 = time.time()
total_2 = t3-t2

Observe the shapes and input and output arrays are matching or not
Input image dimensions: (426, 640, 3)
Output image dimensions: (426, 640, 3)


In [ ]:
input_size = len(content) * 7
output_size = len(code)
print("Input size:", input_size, "(bytes).")
print("Output size:",output_size, "(bytes).")
r = input_size/output_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Thời gian giải nén:", round(total_2,5), "(giây).")

Input size: 33318512 (bytes).
Output size: 16226426 (bytes).
Tỷ số nén: 2.05335
Tỷ lệ nén: 48.7 %
Hiệu suất nén: 51.3 %
Thời gian nén: 1.92614 (giây).
Thời gian giải nén: 6.39445 (giây).


#FANO

In [ ]:
import re
import numpy as np
from PIL import Image
print("Fano Image Compression Program")
print("=================================================================")
import collections
h = int(input("Enter 1 if you want to input an colour image file, 2 for default gray scale case:"))
if h == 1:
    file = input("Enter the filename:")
    res_2 = Image.open(file)
    my_string = np.asarray(Image.open(file),np.uint8)
    sudhi = my_string
    shape = my_string.shape
    #print ("Enetered string is:",my_string)
    message = str(my_string.tolist())
elif h == 2:
    array = np.arange(0, 737280, 1, np.uint8)
    my_string = np.reshape(array, (1024, 720))
    #print ("Enetered string is:",my_string)
    sudhi = my_string
    message = str(my_string.tolist())
else:
    print("You entered invalid input")                        #taking input from user

t0 = time.time()
c = {}
def create_list(message):
    listz = dict(collections.Counter(message))
    # for key, value in list.items():
    #     print(key, ' : ', value)                         #creating the sorted list according to the probablity
    list_sorted = sorted(iter(listz.items()), key = lambda k_v:(k_v[1],k_v[0]),reverse=True)
    final_list = []
    for key,value in list_sorted:
        final_list.append([key,value,''])
    return final_list
def generate(bin_code="", char=[], total=0, new_dict={}):
    if len(char) == 1:
        new_dict[char[0][1]] = bin_code
        return
    count = char[0][0]
    i = 1
    while True:
        left = count + char[i][0]
        if left*2 > total:
            break
        count += char[i][0]
        i+=1
    generate(bin_code+'0',char[:i],count,new_dict)
    generate(bin_code+'1',char[i:],total-count,new_dict)
#print("Shannon tree with merged pathways:")
def divide_list(listz):
    if len(listz) == 2:
        # print([list[0]],"::",[list[1]])               #printing merged pathways
        return [listz[0]],[listz[1]]
    else:
        n = 0
        for i in listz:
            n+= i[1]
        x = 0
        distance = abs(2*x - n)
        j = 0
        for i in range(len(listz)):               #shannon tree structure
            x += listz[i][1]
            if distance < abs(2*x - n):
                j = i
    print(listz[0:j+1],"::",listz[j+1:])               #printing merged pathways
    return listz[0:j+1], listz[j+1:]


def label_list(listz):
    list1,list2 = divide_list(listz)
    for i in list1:
        i[2] += '0'
        c[i[0]] = i[2]
    for i in list2:
        i[2] += '1'
        c[i[0]] = i[2]
    if len(list1)==1 and len(list2)==1:        #assigning values to the tree
        return
    label_list(list2)
    print('c la:')
    print(c)
    return c

# code = label_list(create_list(message))
count = collections.Counter(message)
total = sum(count.values())
char = [x[::-1] for x in list(count.items())]
code = dict()
generate('',char,total,code)
#print("Fano's Encoded Code:")
output = open("compressed_fano.txt","w+")          # generating output binary
letter_binary = []
for key, value in code.items():
    #print(key, ' : ', value)
    letter_binary.append([key,value])
print("Compressed file generated as compressed_fano.txt")
i=0
for a in message:
    for key, value in code.items():
        if key in a:
          #if i<1000:
            #print(key, ' : ', value)
          output.write(value)
          i=i+1
t1 = time.time()
total_1 = t1-t0

t2 = time.time()
output = open("compressed_fano.txt","r")
intermediate = output.readlines()
bitstring = ""
for digit in intermediate:
    bitstring = bitstring + digit
binary ="0b"+bitstring

uncompressed_file_size = len(message)*7
compressed_file_size = len(binary)-2
r = uncompressed_file_size/compressed_file_size
print("Your original file size was", uncompressed_file_size,"(bits). \nThe compressed size is:",compressed_file_size,"(bits).")
print("This is a saving of ",uncompressed_file_size-compressed_file_size,"(bits).")
r = uncompressed_file_size/compressed_file_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Compressed file generated as compressed_fano.txt")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Decoding.......")

uncompressed_string =""
code =""
for digit in bitstring:
    code = code+digit
    pos=0
    for letter in letter_binary:               # decoding the binary and genrating original data
        if code ==letter[1]:
            uncompressed_string=uncompressed_string+letter_binary[pos] [0]
            code=""
        pos+=1

#print("Your UNCOMPRESSED data is:")
if h == 1:
    #temp = re.findall(r'\d+', uncompressed_string)
    #res = list(map(int, temp))
    #res = np.array(res)
    #res = res.astype(np.uint8)
    #res = np.reshape(res, shape)
    #print(res)
    #print("Observe the shapes and input and output arrays are matching or not")
    #print("Input image dimensions:",shape)
    #print("Output image dimensions:",res.shape)
    data = res_2
    data.save('uncompressed_fano.png')
    #if sudhi.all() == res.all():
    print("Success")
    t3 = time.time()
    total_2 = t3-t2
    print("Thời gian giải nén:", round(total_2,5), "(giây).")
if h == 2:
    temp = re.findall(r'\d+', uncompressed_string)
    res = list(map(int, temp))
    print(res)
    res = np.array(res)
    res = res.astype(np.uint8)
    res = np.reshape(res, (1024, 720))
    print(res)
    data = Image.fromarray(res)
    data.save('uncompressed_fano.png')
    print("Success")

Fano Image Compression Program
Enter 1 if you want to input an colour image file, 2 for default gray scale case:1
Enter the filename:5.jpg
Compressed file generated as compressed_fano.txt
Your original file size was 59284232 (bits). 
The compressed size is: 30688090 (bits).
This is a saving of  28596142 (bits).
Tỷ số nén: 1.93183
Tỷ lệ nén: 51.76 %
Hiệu suất nén: 48.24 %
Compressed file generated as compressed_fano.txt
Thời gian nén: 19.73568 (giây).
Decoding.......
Success
Thời gian giải nén: 77.21478 (giây).
